In [1]:
!pip install moviepy

In [2]:
import os
import cv2

def augment_real_data(source_folder):
    # Ensure source folder exists
    if not os.path.exists(source_folder):
        print(f"Source folder '{source_folder}' does not exist.")
        return
    
    # Iterate through all files in the source folder
    for file in os.listdir(source_folder):
        source_path = os.path.join(source_folder, file)
        
        # Check if the file is a video
        if file.endswith(('.mp4', '.avi', '.mov', '.mkv')):
            try:
                # Open the video capture
                cap = cv2.VideoCapture(source_path)
                if not cap.isOpened():
                    print(f"Error opening video file '{file}'.")
                    continue
                
                # Get video properties
                width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                fps = cap.get(cv2.CAP_PROP_FPS)
                
                # Define output file name
                flipped_file_name = "flipped_" + file
                destination_path = os.path.join(source_folder, flipped_file_name)
                
                # Define video writer
                fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # You can change codec as needed
                out = cv2.VideoWriter(destination_path, fourcc, fps, (width, height))
                
                # Read and horizontally flip each frame
                while True:
                    ret, frame = cap.read()
                    if not ret:
                        break
                    flipped_frame = cv2.flip(frame, 1)  # 1 for horizontal flip
                    
                    # Write the flipped frame to the output video
                    out.write(flipped_frame)
                
                # Release resources
                cap.release()
                out.release()
                
                print(f"Successfully flipped '{file}' and saved as '{flipped_file_name}' in '{source_folder}'.")
                
            except Exception as e:
                print(f"Error processing '{file}': {str(e)}")

# Example usage:
source_folder = "real"

augment_real_data(source_folder)


Successfully flipped '000.mp4' and saved as 'flipped_000.mp4' in 'real'.
Successfully flipped '00000.mp4' and saved as 'flipped_00000.mp4' in 'real'.
Successfully flipped '00001.mp4' and saved as 'flipped_00001.mp4' in 'real'.
Successfully flipped '00002.mp4' and saved as 'flipped_00002.mp4' in 'real'.
Successfully flipped '00003.mp4' and saved as 'flipped_00003.mp4' in 'real'.
Successfully flipped '00004.mp4' and saved as 'flipped_00004.mp4' in 'real'.
Successfully flipped '00005.mp4' and saved as 'flipped_00005.mp4' in 'real'.
Successfully flipped '00006.mp4' and saved as 'flipped_00006.mp4' in 'real'.
Successfully flipped '00007.mp4' and saved as 'flipped_00007.mp4' in 'real'.
Successfully flipped '00008.mp4' and saved as 'flipped_00008.mp4' in 'real'.
Successfully flipped '00009.mp4' and saved as 'flipped_00009.mp4' in 'real'.
Successfully flipped '00010.mp4' and saved as 'flipped_00010.mp4' in 'real'.
Successfully flipped '00011.mp4' and saved as 'flipped_00011.mp4' in 'real'.
Suc

In [ ]:
import cv2
import os
import random
import mediapipe as mp
import concurrent.futures

def extract_frames(video_path, output_dir, video_index, num_frames=10, crop_size_factor=1.5, target_size=(256, 256)):
    # Get parent video name
    video_name = os.path.splitext(os.path.basename(video_path))[0]

    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Initialize MediaPipe Face Detection
    mp_face_detection = mp.solutions.face_detection
    mp_drawing = mp.solutions.drawing_utils
    face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.5)

    # Open the video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file '{video_path}'.")
        return

    frame_count = 0
    frames_with_faces = 0

    # Read until 10 frames with faces are found
    while frames_with_faces < num_frames:
        # Capture frame-by-frame
        ret, frame = cap.read()

        if ret:
            # Convert the frame to RGB for MediaPipe
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Detect faces in the frame
            results = face_detection.process(rgb_frame)

            if results.detections:
                for detection in results.detections:
                    bboxC = detection.location_data.relative_bounding_box
                    ih, iw, _ = frame.shape
                    # Expand the bounding box to create a larger ROI
                    x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), \
                                 int(bboxC.width * iw), int(bboxC.height * ih)
                    cx, cy = x + w // 2, y + h // 2  # Center of the original face bounding box
                    nw, nh = int(w * crop_size_factor), int(h * crop_size_factor)  # New width and height
                    # New coordinates for the expanded bounding box
                    nx, ny = max(0, cx - nw // 2), max(0, cy - nh // 2)
                    nx2, ny2 = min(iw, nx + nw), min(ih, ny + nh)
                    # Crop the face region from the frame
                    face_crop = frame[ny:ny2, nx:nx2]   

                    # Resize the cropped face region to the target size
                    resized_face_crop = cv2.resize(face_crop, target_size)

                    # Save the resized face as an image file
                    frame_path = os.path.join(output_dir, f"video{video_index}_frame{frame_count}.jpg")
                    cv2.imwrite(frame_path, resized_face_crop)

                    # Increment frame count and frames with faces count
                    frame_count += 1
                    frames_with_faces += 1
                    break  # Only consider the first face
        else:
            break

    # Release resources
    cap.release()

    print(f"{frames_with_faces} frames with faces extracted from '{video_path}'")

def extract_frames_from_directory_parallel(input_dir, output_dir, num_frames=10, crop_size_factor=1.5, target_size=(256, 256)):
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # List video files
    video_files = [f for f in os.listdir(input_dir) if f.endswith('.mp4')]
    num_videos = len(video_files)

    # Process videos in parallel
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(extract_frames, os.path.join(input_dir, video), output_dir, index, num_frames, crop_size_factor, target_size)
                   for index, video in enumerate(video_files, start=1)]

        # Wait for all futures to complete
        for future in concurrent.futures.as_completed(futures):
            try:
                future.result()
            except Exception as e:
                print(f"Error processing video: {e}")

# Example usage:
input_directory = "fake"
output_directory = "frames_with_faces"
num_frames = 150
crop_size_factor = 1.5  
target_size = (256, 256)  
extract_frames_from_directory_parallel(input_directory, output_directory, num_frames, crop_size_factor, target_size)


150 frames with faces extracted from 'fake\celebfake_id0_id1_0007.mp4'
150 frames with faces extracted from 'fake\celebfake_id0_id17_0001.mp4'
150 frames with faces extracted from 'fake\celebfake_id0_id28_0004.mp4'
150 frames with faces extracted from 'fake\celebfake_id0_id28_0001.mp4'
150 frames with faces extracted from 'fake\celebfake_id0_id1_0001.mp4'
150 frames with faces extracted from 'fake\celebfake_id0_id23_0005.mp4'
150 frames with faces extracted from 'fake\celebfake_id0_id26_0008.mp4'
150 frames with faces extracted from 'fake\celebfake_id0_id26_0007.mp4'
150 frames with faces extracted from 'fake\celebfake_id0_id28_0005.mp4'
150 frames with faces extracted from 'fake\celebfake_id0_id21_0004.mp4'
150 frames with faces extracted from 'fake\celebfake_id0_id21_0001.mp4'
150 frames with faces extracted from 'fake\celebfake_id0_id28_0008.mp4'
150 frames with faces extracted from 'fake\celebfake_id0_id21_0007.mp4'
150 frames with faces extracted from 'fake\celebfake_id0_id17_0007

In [ ]:
import os
import cv2
import re
from collections import defaultdict

def combine_frames_to_video(input_path, output_path):
    # Create output directory if it doesn't exist
    os.makedirs(output_path, exist_ok=True)

    # Iterate through each file in the input folder
    video_files = defaultdict(list)

    for filename in os.listdir(input_path):
        if filename.endswith('.jpg'):
            match = re.match(r'(video\d+)_frame(\d+)\.jpg', filename)
            if match:
                video_name, frame_number = match.groups()
                video_files[video_name].append((int(frame_number), filename))

    for video_name, frames in video_files.items():
        # Sort frames based on their sequence number
        frames.sort(key=lambda x: x[0])

        # Create video writer
        output_filename = f'{video_name}.mp4'
        output_filepath = os.path.join(output_path, output_filename)
        frame = cv2.imread(os.path.join(input_path, frames[0][1]))
        height, width, _ = frame.shape
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_filepath, fourcc, 30, (width, height))

        # Combine frames into video
        for _, frame_name in frames:
            frame = cv2.imread(os.path.join(input_path, frame_name))
            out.write(frame)

        # Release resources
        out.release()
        print(f"Video created: {output_filepath}")

# Example usage:
input_path = "frames_with_faces"  # Input directory containing frames
output_path = "output_videos"  # Output directory to save the combined videos
combine_frames_to_video(input_path, output_path)


Video created: output_videos\video1000.mp4
Video created: output_videos\video1001.mp4
Video created: output_videos\video1002.mp4
Video created: output_videos\video1003.mp4
Video created: output_videos\video1004.mp4
Video created: output_videos\video1005.mp4
Video created: output_videos\video1006.mp4
Video created: output_videos\video1007.mp4
Video created: output_videos\video1008.mp4
Video created: output_videos\video1009.mp4
Video created: output_videos\video100.mp4
Video created: output_videos\video1010.mp4
Video created: output_videos\video1011.mp4
Video created: output_videos\video1012.mp4
Video created: output_videos\video1013.mp4
Video created: output_videos\video1014.mp4
Video created: output_videos\video1015.mp4
Video created: output_videos\video1016.mp4
Video created: output_videos\video1017.mp4
Video created: output_videos\video1018.mp4
Video created: output_videos\video1019.mp4
Video created: output_videos\video101.mp4
Video created: output_videos\video1020.mp4
Video created

In [ ]:
import cv2
import os
import random
import mediapipe as mp
import concurrent.futures

def extract_frames(video_path, output_dir, video_index, num_frames=10, crop_size_factor=1.5, target_size=(256, 256)):
    # Get parent video name
    video_name = os.path.splitext(os.path.basename(video_path))[0]

    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Initialize MediaPipe Face Detection
    mp_face_detection = mp.solutions.face_detection
    mp_drawing = mp.solutions.drawing_utils
    face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.5)

    # Open the video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file '{video_path}'.")
        return

    frame_count = 0
    frames_with_faces = 0

    # Read until 10 frames with faces are found
    while frames_with_faces < num_frames:
        # Capture frame-by-frame
        ret, frame = cap.read()

        if ret:
            # Convert the frame to RGB for MediaPipe
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Detect faces in the frame
            results = face_detection.process(rgb_frame)

            if results.detections:
                for detection in results.detections:
                    bboxC = detection.location_data.relative_bounding_box
                    ih, iw, _ = frame.shape
                    # Expand the bounding box to create a larger ROI
                    x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), \
                                 int(bboxC.width * iw), int(bboxC.height * ih)
                    cx, cy = x + w // 2, y + h // 2  # Center of the original face bounding box
                    nw, nh = int(w * crop_size_factor), int(h * crop_size_factor)  # New width and height
                    # New coordinates for the expanded bounding box
                    nx, ny = max(0, cx - nw // 2), max(0, cy - nh // 2)
                    nx2, ny2 = min(iw, nx + nw), min(ih, ny + nh)
                    # Crop the face region from the frame
                    face_crop = frame[ny:ny2, nx:nx2]

                    # Resize the cropped face region to the target size
                    resized_face_crop = cv2.resize(face_crop, target_size)

                    # Save the resized face as an image file
                    frame_path = os.path.join(output_dir, f"video{video_index}_frame{frame_count}.jpg")
                    cv2.imwrite(frame_path, resized_face_crop)

                    # Increment frame count and frames with faces count
                    frame_count += 1
                    frames_with_faces += 1
                    break  # Only consider the first face
        else:
            break

    # Release resources
    cap.release()

    print(f"{frames_with_faces} frames with faces extracted from '{video_path}'")

def extract_frames_from_directory_parallel(input_dir, output_dir, num_frames=10, crop_size_factor=1.5, target_size=(256, 256)):
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # List video files
    video_files = [f for f in os.listdir(input_dir) if f.endswith('.mp4')]
    num_videos = len(video_files)

    # Process videos in parallel
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(extract_frames, os.path.join(input_dir, video), output_dir, index, num_frames, crop_size_factor, target_size)
                   for index, video in enumerate(video_files, start=1)]

        # Wait for all futures to complete
        for future in concurrent.futures.as_completed(futures):
            try:
                future.result()
            except Exception as e:
                print(f"Error processing video: {e}")

# Example usage:
input_directory = "real"
output_directory = "frames_with_faces_real"
num_frames = 150
crop_size_factor = 1.5  
target_size = (256, 256)  
extract_frames_from_directory_parallel(input_directory, output_directory, num_frames, crop_size_factor, target_size)


150 frames with faces extracted from 'real\000.mp4'
150 frames with faces extracted from 'real\00013.mp4'
150 frames with faces extracted from 'real\00000.mp4'
150 frames with faces extracted from 'real\00002.mp4'
150 frames with faces extracted from 'real\00004.mp4'
150 frames with faces extracted from 'real\00007.mp4'
150 frames with faces extracted from 'real\00011.mp4'
150 frames with faces extracted from 'real\00016.mp4'
150 frames with faces extracted from 'real\00006.mp4'
150 frames with faces extracted from 'real\00017.mp4'150 frames with faces extracted from 'real\00012.mp4'

150 frames with faces extracted from 'real\00001.mp4'
150 frames with faces extracted from 'real\00018.mp4'
150 frames with faces extracted from 'real\00009.mp4'
150 frames with faces extracted from 'real\00003.mp4'
150 frames with faces extracted from 'real\00015.mp4'
150 frames with faces extracted from 'real\00010.mp4'
150 frames with faces extracted from 'real\00014.mp4'
150 frames with faces extracte

In [1]:
import os
import cv2
import re
from collections import defaultdict

def combine_frames_to_video(input_path, output_path):
    # Create output directory if it doesn't exist
    os.makedirs(output_path, exist_ok=True)

    # Iterate through each file in the input folder
    video_files = defaultdict(list)

    for filename in os.listdir(input_path):
        if filename.endswith('.jpg'):
            match = re.match(r'(video\d+)_frame(\d+)\.jpg', filename)
            if match:
                video_name, frame_number = match.groups()
                video_files[video_name].append((int(frame_number), filename))

    for video_name, frames in video_files.items():
        # Sort frames based on their sequence number
        frames.sort(key=lambda x: x[0])

        # Create video writer
        output_filename = f'{video_name}.mp4'
        output_filepath = os.path.join(output_path, output_filename)
        frame = cv2.imread(os.path.join(input_path, frames[0][1]))
        height, width, _ = frame.shape
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_filepath, fourcc, 30, (width, height))

        # Combine frames into video
        for _, frame_name in frames:
            frame = cv2.imread(os.path.join(input_path, frame_name))
            out.write(frame)

        # Release resources
        out.release()
        print(f"Video created: {output_filepath}")

# Example usage:
input_path = "frames_with_faces_real"  # Input directory containing frames
output_path = "output_videos_real"  # Output directory to save the combined videos
combine_frames_to_video(input_path, output_path)


Video created: output_videos_real\video1000.mp4
Video created: output_videos_real\video1001.mp4
Video created: output_videos_real\video1002.mp4
Video created: output_videos_real\video1003.mp4
Video created: output_videos_real\video1004.mp4
Video created: output_videos_real\video1005.mp4
Video created: output_videos_real\video1006.mp4
Video created: output_videos_real\video1007.mp4
Video created: output_videos_real\video1008.mp4
Video created: output_videos_real\video1009.mp4
Video created: output_videos_real\video100.mp4
Video created: output_videos_real\video1010.mp4
Video created: output_videos_real\video1011.mp4
Video created: output_videos_real\video1012.mp4
Video created: output_videos_real\video1013.mp4
Video created: output_videos_real\video1014.mp4
Video created: output_videos_real\video1015.mp4
Video created: output_videos_real\video1016.mp4
Video created: output_videos_real\video1017.mp4
Video created: output_videos_real\video1018.mp4
Video created: output_videos_real\video10

Video created: output_videos_real\video1000.mp4
Video created: output_videos_real\video1001.mp4
Video created: output_videos_real\video1002.mp4
Video created: output_videos_real\video1003.mp4
Video created: output_videos_real\video1004.mp4
Video created: output_videos_real\video1005.mp4
Video created: output_videos_real\video1006.mp4
Video created: output_videos_real\video1007.mp4
Video created: output_videos_real\video1008.mp4
Video created: output_videos_real\video1009.mp4
Video created: output_videos_real\video100.mp4
Video created: output_videos_real\video1010.mp4
Video created: output_videos_real\video1011.mp4
Video created: output_videos_real\video1012.mp4


KeyboardInterrupt: 